# Final Project 

In [1]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi, cos, sin, exp, sqrt
from scipy.signal import freqz, welch, periodogram, butter, lfilter, filtfilt, boxcar
from textwrap import wrap
from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import and clean up the data

In [33]:
# Import data
AS = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=1, usecols=[0,35])

DC = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DeepCove_temperature.dat', 
                 header=2)

DE = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DiscoveryElementary_temperature.dat', 
                 header=2)

HL = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Helgesen_temperature.dat', 
                 header=2)

JB = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JamesBay_temperature.dat', 
                 header=2)

JM = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JohnMuir_temperature.dat', 
                 header=2)

KT = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Keating_temperature.dat', 
                 header=2)

US = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/UVicSci_temperature.dat', 
                 header=2)

### Clean up the All Stations data

In [10]:
# Convert times from MatLab time to Python Time
AS['Time'] = AS['NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
AS2 = AS.rename(index=str, 
                columns={"NaN": "MatLab Time", "48.4623": "Temperature"})

# Reorder columns 
cols = AS2.columns.tolist()
cols = cols[-1:] + cols[:-1]
AS3 = AS2[cols]

# Set time as index column
DatAS = AS3.set_index('Time')

### Clean up the minute resolution data

In [27]:
# Generate dates
date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     end='2017-08-30 16:59:00.000000', 
                     freq='min')

def DateInsert(Data):
    D2 = Data
    # Insert dates into D2 dataframe
    D2.insert(loc=0, column='Time', value=date)
    # Rename the columns
    D22 = D2.rename(index=str, columns={"2979360": "Temperature"})
    # Set index
    DM = D22.set_index('Time')
    return DM

In [28]:
DataM = [DC, DE, HL, JB, JM, KT, US]

In [29]:
DataMIns = [DateInsert(DataM[i]) for i in range(0, len(DataM))]

ValueError: Length of values does not match length of index

In [34]:
DataM[0]

,Time,2979360
0,2011-12-31 17:00:00,4.44
1,2011-12-31 17:01:00,4.40
2,2011-12-31 17:02:00,4.39
3,2011-12-31 17:03:00,4.39
4,2011-12-31 17:04:00,4.33
5,2011-12-31 17:05:00,4.33
6,2011-12-31 17:06:00,4.33
7,2011-12-31 17:07:00,4.30
8,2011-12-31 17:08:00,4.28
9,2011-12-31 17:09:00,4.28
